In [1]:
import polars as pl
from datetime import datetime
from typing import Tuple, List
import warnings
import sys
import numpy as np
import scipy
import statsmodels.api as sm
from alpha101_prod import CalcAlpha101Factor

POS_RET_PCT_SCALE_THRESHOLD = 0.01

input_path = "data/all_data_1d.parquet"
output_path = "data/predictions.parquet"
is_prod = False

In [2]:
def undimensionalize(data, column_name, window = 50):
    data = data.with_columns((pl.col(column_name) / pl.col(column_name).rolling_mean(window) - 1).over('symbol').alias(column_name))
    return data

def standardize_z(data, column_name):
    data = data.with_columns(((pl.col(column_name) - pl.col(column_name).mean()) / pl.col(column_name).std()).over('open_time')
        .alias(column_name))
    return data
def sign(column):
    return pl.when(column > 0).then(1).otherwise(pl.when(column < 0).then(-1).otherwise(0))

In [3]:
def AddPastReturnFactor(input_df: pl.DataFrame, day_num: int) -> pl.DataFrame:
    input_df = input_df.sort(["symbol", "open_time"])
    # 千万注意，这里是计算此时刻相对于前一时刻的return，不能使用未来信息 -> shift(1)是整体往下移动
    for i in range(1, day_num + 1):
        input_df = input_df.with_columns(
            ((pl.col("close") / pl.col("close").shift(i) - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"past_{i}day_close_return")
        )

    # 默认使用过去1天的return作为return列
    input_df = input_df.with_columns(pl.col("past_1day_close_return").alias("return"))

    return input_df

In [4]:
def CalcDayPositionScale(
    input_df: pl.DataFrame, day_num: int, trade_long_rank: int, trade_short_rank: int
) -> pl.DataFrame:
    # print(input_df["linear_compound_factor_1day"])
    agg_avg_ret_list = []
    for i in range(1, day_num + 1):
        for side in ["long", "short"]:
            sort_desc = True if side == "long" else False
            trade_rank_num = trade_long_rank if side == "long" else trade_short_rank

            input_df = input_df.with_columns(
                pl.col(f"linear_compound_factor_{i}day")
                .rank(descending=sort_desc)
                .over("open_time")
                .alias(f"symbol_rank_{side}_{i}day")
            ).with_columns(
                pl.when(pl.col(f"symbol_rank_{side}_{i}day") <= trade_rank_num)
                .then(pl.col(f"close_price_fut_{i}day_ret"))
                .otherwise(None)
                .alias(f"total_{side}_value_scale_{i}day")
            )

            cur_agg_avg_ret = (
                input_df.group_by("open_time")
                .agg(
                    pl.col(f"total_{side}_value_scale_{i}day")
                    .mean()
                    .alias(f"fut_mean_{side}_ret_{i}day"),
                )
                .sort("open_time")
            )
            agg_avg_ret_list.append(cur_agg_avg_ret)

    agg_avg_ret_df = agg_avg_ret_list[0]
    for df in agg_avg_ret_list[1:]:
        agg_avg_ret_df = agg_avg_ret_df.join(df, on="open_time", how="right")

    # Sort the final DataFrame by open_time
    agg_avg_ret_df = agg_avg_ret_df.sort("open_time")
    agg_avg_ret_df = agg_avg_ret_df.select(
        pl.col(
            ["open_time"]
            + [col for col in agg_avg_ret_df.columns if col != "open_time"]
        )
    )

    for i in range(1, day_num + 1):
        for side in ["long", "short"]:
            agg_avg_ret_df = agg_avg_ret_df.with_columns(
                pl.col(f"fut_mean_{side}_ret_{i}day")
                .shift(i)
                .alias(f"past_mean_{side}_ret_{i}day"),
            )

    for i in range(1, day_num + 1):
        for side in ["long", "short"]:
            bullish_scale = 1.2 if side == "long" else 0.8
            bearish_scale = 0.8 if side == "long" else 1.2
            agg_avg_ret_df = agg_avg_ret_df.with_columns(
                pl.when(
                    (pl.col(f"past_mean_long_ret_{i}day") > POS_RET_PCT_SCALE_THRESHOLD)
                    & (
                        pl.col(f"past_mean_short_ret_{i}day")
                        > POS_RET_PCT_SCALE_THRESHOLD
                    )
                )
                .then(bullish_scale)
                .when(
                    (pl.col(f"past_mean_long_ret_{i}day") < POS_RET_PCT_SCALE_THRESHOLD)
                    & (
                        pl.col(f"past_mean_short_ret_{i}day")
                        < POS_RET_PCT_SCALE_THRESHOLD
                    )
                )
                .then(bearish_scale)
                .otherwise(1.0)
                .alias(f"{side}_value_scale_{i}day")
            )
    return input_df, agg_avg_ret_df


In [5]:
def AddTotalPosValueScale(
    input_df: pl.DataFrame, day_num: int, trade_long_rank: int, trade_short_rank: int
) -> pl.DataFrame:
    _, day_scale_df = CalcDayPositionScale(
        input_df, day_num, trade_long_rank, trade_short_rank
    )

    # only need the scale columns
    select_col = [col for col in day_scale_df.columns if "scale" in col]
    day_scale_df = day_scale_df.select(pl.col(["open_time"] + select_col))

    # print(f"before join: {input_df}")
    # print(f"day scale df: ", day_scale_df)

    input_df = input_df.join(day_scale_df, on="open_time", how="left")

    # print(f"after join     : {input_df}")
    return input_df, day_scale_df


In [6]:
def AddAmihud(input_df: pl.DataFrame, window_size: int = 10) -> pl.DataFrame:
    # Calculate rolling sums for absolute returns and quote volume
    input_df = input_df.with_columns(
        pl.col("return").abs().rolling_sum(window_size=window_size).over("symbol").alias("rolling_abs_return_sum"),
        pl.col("quote_volume").rolling_sum(window_size=window_size).over("symbol").alias("rolling_quote_volume_sum"),
    )

    # Calculate Amihud illiquidity measure
    input_df = input_df.with_columns(
        (pl.col("rolling_abs_return_sum") / pl.col("rolling_quote_volume_sum"))
        .over("symbol")  # Apply the final operation within each symbol group
        .alias("amihud")
    )

    # Drop intermediate columns
    input_df = input_df.drop(["rolling_abs_return_sum", "rolling_quote_volume_sum"])

    return input_df

In [7]:
def AddReturnAutocorr(input_df: pl.DataFrame, window_size: int = 20, lag: int = 1) -> pl.DataFrame:
    # Calculate rolling sums for absolute returns and quote volume
    input_df = input_df.with_columns(
        pl.rolling_corr(pl.col("return"), pl.col("return").shift(1), window_size=window_size).over("symbol").alias("return_autocorr_" + str(lag))
    )
    return input_df

In [8]:
def AddReturnSkewness(input_df: pl.DataFrame, window_size: int = 20) -> pl.DataFrame:
    # Calculate rolling sums for absolute returns and quote volume
    input_df = input_df.with_columns(
        pl.col("return").rolling_skew(window_size).over("symbol").alias("return_skewness")
    )
    return input_df

In [9]:
def AddTakerBuyRatio(input_df: pl.DataFrame) -> pl.DataFrame:
    input_df = input_df.with_columns(
        (pl.col("taker_buy_volume") / pl.col("volume")).over('symbol').alias("taker_buy_ratio")
    )
    return input_df

In [10]:
def AddID(input_df: pl.DataFrame) -> pl.DataFrame:
    input_df = input_df.with_columns((pl.col('close') / pl.col('open') - 1).over('symbol').alias('ret1'))
    input_df = input_df.sort(by=['symbol', 'open_time'])

    # 计算过去 3天的最大涨幅和最小涨幅
    max_return = pl.col("ret1").rolling_max(window_size=48)
    min_return = pl.col("ret1").rolling_min(window_size=48)
    # 计算 ID
    input_df = input_df.with_columns(
        (sign(pl.col('ret1'))
        * (pl.col("ret1").rolling_max(window_size=48) - pl.col("ret1").rolling_min(window_size=72)) / pl.col('close'))
        .over('symbol')
        .alias("ID")
    )
    input_df = input_df.with_columns(
        pl.col('ID').clip(-0.2, 0.3).alias('ID')
    )
    input_df = input_df.drop('ret1')
    return input_df

In [11]:
def AddAutocorrRank(input_df: pl.DataFrame, window_size: int = 3) -> pl.DataFrame:
    input_df = input_df.with_columns(
        pl.col('close').shift(2).over('symbol').alias('shift_close2')
    )
    input_df = input_df.with_columns([
        pl.col("close").rank().over("symbol").alias("rank_close"),
        pl.col("shift_close2").rank().over("symbol").alias("rank_shift_close2")
    ])
    # 计算排名后的滚动Pearson相关系数，实际上就是滚动的Spearman相关系数
    input_df = input_df.with_columns(
        pl.rolling_corr("rank_close", "rank_shift_close2", window_size=6)
        .over("symbol")
        .alias("spearman_corr2_w3")
    )
    input_df = input_df.drop(["shift_close2","rank_close","rank_shift_close2"])
    return input_df

In [12]:
def beat_ratio(input_df: pl.DataFrame) -> pl.DataFrame:
    input_df = input_df.with_columns([
    (pl.col("quote_volume") - pl.col('taker_buy_quote_volume')).alias("taker_sell_quote_volume")
    ])

    input_df = input_df.with_columns([
        (pl.col("taker_buy_quote_volume") / pl.col("taker_sell_quote_volume")).alias("beat_ratio")
    ])

    return input_df

In [13]:
def factor011(input_df: pl.DataFrame, x1: int, x2: int) -> pl.DataFrame:
    
    input_df = input_df.with_columns([
        (pl.col('quote_volume') / pl.col('volume')).alias('vwap'),
        (pl.col('volume') - (pl.col('volume').shift(x1).over('symbol'))).alias('volume_change')
    ])

    input_df = input_df.with_columns([
        (pl.col('vwap') - pl.col('close')).rolling_max(x2).over('symbol').alias('vwap_close_max'),
        (pl.col('vwap') - pl.col('close')).rolling_min(x2).over('symbol').alias('vwap_close_min')
    ])

    input_df = input_df.with_columns([
        pl.col('vwap_close_max').rank('dense').over('open_time').alias('vwap_close_max_rank'),
        pl.col('vwap_close_min').rank('dense').over('open_time').alias('vwap_close_min_rank'),
        pl.col('volume_change').rank('dense').over('open_time').alias('volume_change_rank')
    ])

    input_df = input_df.with_columns([
        (pl.col('vwap_close_max_rank') + pl.col('vwap_close_min_rank') * pl.col('volume_change_rank')).alias('factor011_rank')
    ])

    return input_df

In [14]:
def nettotal_taker_quote_volume(input_df: pl.DataFrame, N: int) -> pl.DataFrame:

    input_df = input_df.with_columns([
        (pl.col('quote_volume') - pl.col('taker_buy_quote_volume')).alias('taker_sell_quote_volume')
    ])

    input_df = input_df.with_columns([
        pl.col('taker_buy_quote_volume').rolling_sum(N).over('symbol').alias('taker_buy_quote_volume_sum'),
        pl.col('taker_sell_quote_volume').rolling_sum(N).over('symbol').alias('taker_sell_quote_volume_sum'),
    ])

    input_df = input_df.with_columns([
        (pl.col('taker_buy_quote_volume_sum') - pl.col('taker_sell_quote_volume_sum')).alias('net_taker_quote_volume'),
        (pl.col('taker_buy_quote_volume_sum') + pl.col('taker_sell_quote_volume_sum')).alias('total_taker_quote_volume')
    ])

    return input_df

In [15]:
def AddFutureRetCol(input_df: pl.DataFrame, day_num: int) -> pl.DataFrame:
    input_df = input_df.sort(["symbol", "open_time"])
    for i in range(1, day_num + 1):
        input_df = input_df.with_columns(
            ((pl.col("close").shift(-i) / pl.col("close") - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"close_price_fut_{i}day_ret")
        )
        input_df = input_df.with_columns(
            ((pl.col("open").shift(-i) / pl.col("open") - 1) * 100)
            .over("symbol")  # Applying the function over each symbol group
            .alias(f"open_price_fut_{i}day_ret")
        )
    return input_df

In [16]:
def fama_macbeth_get_factor_weight(
    train_data: pl.DataFrame,
    update_pos_days: int,
    factor_num: int,
    factor_combination_list: List[str],
) -> Tuple[np.ndarray, float]:
    # Drop rows containing any null values
    train_data = train_data.drop_nulls()

    total_weights_sum = np.zeros(factor_num)
    unique_times = train_data.select(pl.col("open_time").sort()).unique().to_numpy()

    constant_sum = 0.0

    for each_time in unique_times:
        y_column_name = f"close_price_fut_{update_pos_days}day_ret"
        assert (
            y_column_name in train_data.columns
        ), f"Column {y_column_name} (as y) not found in train data"

        slice_data = train_data.filter(pl.col("open_time") == each_time).fill_nan(0)

        X = slice_data[factor_combination_list].to_numpy()
        X = sm.add_constant(X)  # Add constant term (intercept)
        y = slice_data[y_column_name].to_numpy()

        model = sm.OLS(y, X)
        results = model.fit()
        weights = results.params[1:]
        # print(results.params[1:])
        constant_sum += results.params[0]  # constant term

        while weights.shape[0] < total_weights_sum.shape[0]:
            weights = np.append(weights, 0)

        total_weights_sum += weights
        print(total_weights_sum)

    total_weights_sum /= len(unique_times)
    avg_const_term = constant_sum / len(unique_times)

    return total_weights_sum, avg_const_term


In [17]:
def CalcLinearCompoundFactor(
    input_df: pl.DataFrame,
    day_num: int,
    factor_combination_list: list,
    date_threshold: datetime = datetime(2023, 1, 1),
) -> pl.DataFrame:

    factor_num = len(factor_combination_list)

    for cur_update_position_time in range(1, day_num + 1):
        cur_fut_ret_column_name = f"close_price_fut_{cur_update_position_time}day_ret"

        non_nan_result = input_df.filter(
            (pl.col(cur_fut_ret_column_name).is_not_nan())
            & (pl.col(cur_fut_ret_column_name).is_not_null())
        ).sort(["open_time", "symbol"])

        non_nan_linear_x = non_nan_result.select(
            ["open_time", "symbol", cur_fut_ret_column_name] + factor_combination_list
        )

        date_threshold_ms = date_threshold.timestamp() * 1000
        linear_x_train = non_nan_linear_x.filter(
            pl.col("open_time") < date_threshold_ms
        )

        # # 检查过确实只使用训练数据拟合，没有用测试信息
        # print(linear_x_train.sort(by="open_time"))

        weighted_factors, const_term = fama_macbeth_get_factor_weight(
            linear_x_train,
            cur_update_position_time,
            factor_num=factor_num,
            factor_combination_list=factor_combination_list,
        )

        weighted_sum_expr = pl.lit(const_term)
        for factor, weight in zip(factor_combination_list, weighted_factors):
            weighted_sum_expr += pl.col(factor) * weight

        # 这个代码用于实际交易的时候，我们只需要使用权重计算未来收益率
        input_df = input_df.with_columns(
            weighted_sum_expr.alias(
                f"linear_compound_factor_{cur_update_position_time}day"
            )
        )

    return input_df.filter(
        pl.col("open_time") >= date_threshold_ms
    )  # only return the data after the threshold


In [18]:
def AddVolatilityCol(input_df: pl.DataFrame) -> pl.DataFrame:
    def calculate_volatility(group, bar_name: str, window_size: int = 30):
        return group.with_columns(
            [
                pl.col(bar_name)
                .pct_change()
                .rolling_std(window_size=window_size)
                .alias(f"{bar_name}_price_volatility")
            ]
        )

    input_df = input_df.group_by("symbol").map_groups(
        lambda x: calculate_volatility(x, "open", window_size=30)
    )
    input_df = input_df.group_by("symbol").map_groups(
        lambda x: calculate_volatility(x, "close", window_size=30)
    )
    return input_df

In [19]:
FACTOR_COMBINATION_LIST = [
    "amihud",
    "return_skewness",
    "alpha30",
    "alpha36",
    "alpha40",
    "alpha45",
    'net_taker_quote_volume',
]
UPDATE_POSITION_TIME = 10

input_data = pl.read_parquet(input_path)
input_data = input_data.with_columns(pl.from_epoch(pl.col("open_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("open_time"))
input_data = input_data.with_columns(pl.from_epoch(pl.col("close_time"), time_unit="ms").cast(pl.Datetime('ms')).alias("close_time"))
input_data = input_data.sort(by=["symbol", "open_time"])

# for production need
input_data = input_data.filter(
    ~pl.col("symbol").is_in(
        ["BTCUSDT", "ETHUSDT", "BCHUSDT", "LTCUSDT", "ETCUSDT", "LINKUSDT", "AVAXUSDT", "SOLUSDT"]
    )
).filter(pl.col("symbol").str.ends_with("USDT"))

input_data = AddPastReturnFactor(input_data, day_num=10)
input_data = AddVolatilityCol(input_data)
input_data = AddAmihud(input_data)
# input_data = AddReturnAutocorr(input_data, 28, 1)
input_data = AddReturnSkewness(input_data, 7)
# input_data = AddTakerBuyRatio(input_data)  # 1.666
# input_data = AddID(input_data)  # 1.895
# input_data = AddAutocorrRank(input_data)  # 1.371
# input_data = beat_ratio(input_data)  # 1.540
# input_data = factor011(input_data, 60, 65)  # 1.980
input_data = nettotal_taker_quote_volume(input_data, 20)  # 1.634

# Calculate alpha101 factor
alpha101_factor_list = [x for x in FACTOR_COMBINATION_LIST if "alpha" in x]
input_data = CalcAlpha101Factor(input_data, calc_factor_list=alpha101_factor_list)

input_data

get alpah func alpha30
get alpah func alpha36
get alpah func alpha40
get alpah func alpha45
alpha30
alpha36
alpha40
alpha45


symbol,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,past_1day_close_return,past_2day_close_return,past_3day_close_return,past_4day_close_return,past_5day_close_return,past_6day_close_return,past_7day_close_return,past_8day_close_return,past_9day_close_return,past_10day_close_return,return,open_price_volatility,close_price_volatility,amihud,return_skewness,taker_sell_quote_volume,taker_buy_quote_volume_sum,taker_sell_quote_volume_sum,net_taker_quote_volume,total_taker_quote_volume,alpha30,alpha36,alpha40,alpha45
str,datetime[ms],f64,f64,f64,f64,f64,datetime[ms],f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""DODOXUSDT""",2023-08-08 00:00:00,0.1358,0.1594,0.1212,0.1281,8.99742803e8,2023-08-08 23:59:59.999,1.2496e8,492553,4.30200951e8,5.9693e7,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.5268e7,null,null,null,null,0.0,0.0,0.0,0.0
"""DODOXUSDT""",2023-08-09 00:00:00,0.1282,0.1616,0.1054,0.118,1.6340e9,2023-08-09 23:59:59.999,2.0207e8,750869,8.07491597e8,1.0001e8,-7.884465,null,null,null,null,null,null,null,null,null,-7.884465,null,null,null,null,1.0206e8,null,null,null,null,0.0,0.0,0.0,0.0
"""DODOXUSDT""",2023-08-10 00:00:00,0.118,0.1182,0.1062,0.1129,4.56935509e8,2023-08-10 23:59:59.999,5.1152e7,207948,2.22160545e8,2.4884e7,-4.322034,-11.86573,null,null,null,null,null,null,null,null,-4.322034,null,null,null,null,2.6268e7,null,null,null,null,0.0,0.0,0.0,0.0
"""DODOXUSDT""",2023-08-11 00:00:00,0.1129,0.1176,0.1056,0.116,5.33818723e8,2023-08-11 23:59:59.999,6.0821e7,273080,2.63423901e8,3.0046e7,2.745793,-1.694915,-9.445746,null,null,null,null,null,null,null,2.745793,null,null,null,null,3.0775e7,null,null,null,null,0.0,0.0,0.0,0.0
"""DODOXUSDT""",2023-08-12 00:00:00,0.116,0.122,0.1115,0.1159,3.23080679e8,2023-08-12 23:59:59.999,3.7238e7,160874,1.61328669e8,1.8598e7,-0.086207,2.657219,-1.779661,-9.52381,null,null,null,null,null,null,-0.086207,null,null,null,null,1.8640e7,null,null,null,null,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""COSUSDT""",2024-10-30 00:00:00,0.007197,0.00744,0.007013,0.007208,7.23297668e8,2024-10-30 23:59:59.999,5.2149e6,60823,3.59555962e8,2.5925e6,0.125017,3.667482,5.180213,7.102526,7.855753,0.839396,3.429473,-1.530055,-1.219679,-4.174422,0.125017,0.037107,0.03322,8.8784e-7,-1.548216,2.6223e6,4.3295e7,4.9204e7,-5.9084e6,9.2499e7,0.006866,0.0,-0.03531,-0.049913
"""COSUSDT""",2024-10-31 00:00:00,0.007202,0.007229,0.006827,0.006882,4.66377331e8,2024-10-31 23:59:59.999,3.2641e6,43832,2.14896023e8,1.5036e6,-4.522752,-4.403389,-1.021142,0.423172,2.258544,2.977705,-3.721321,-1.248386,-5.983607,-5.687269,-4.522752,0.033113,0.031126,9.6322e-7,-0.736953,1.7605e6,4.2145e7,4.7836e7,-5.6905e6,8.9981e7,0.019401,0.0,-0.032497,-0.045078
"""COSUSDT""",2024-11-01 00:00:00,0.00688,0.007026,0.006704,0.006848,2.71284834e8,2024-11-01 23:59:59.999,1.8568e6,30902,1.25900771e8,861717.804583,-0.494042,-4.994451,-4.875677,-1.51014,-0.072961,1.753343,2.468951,-4.196978,-1.736261,-6.448087,-0.494042,0.031036,0.028714,9.9853e-7,-0.922395,995059.589658,4.0038e7,4.5519e7,-5.4812e6,8.5558e7,0.036927,0.0,-0.032844,-0.009364


In [20]:
# 截面归一化
input_data = input_data.sort(by=["symbol", "open_time"])
original_columns = input_data.columns
for c in FACTOR_COMBINATION_LIST:
    print(c)
    input_data = input_data.with_columns(pl.col(c).mean().over("open_time").alias("mean_" + c))
    input_data = input_data.with_columns(pl.col(c).std().over("open_time").alias("std_" + c))
    input_data = input_data.with_columns(((pl.col(c) - pl.col("mean_" + c)) / pl.col("std_" + c)).alias(c))
input_data = input_data.select(original_columns)
input_data


amihud
return_skewness
alpha30
alpha36
alpha40
alpha45
net_taker_quote_volume


symbol,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,past_1day_close_return,past_2day_close_return,past_3day_close_return,past_4day_close_return,past_5day_close_return,past_6day_close_return,past_7day_close_return,past_8day_close_return,past_9day_close_return,past_10day_close_return,return,open_price_volatility,close_price_volatility,amihud,return_skewness,taker_sell_quote_volume,taker_buy_quote_volume_sum,taker_sell_quote_volume_sum,net_taker_quote_volume,total_taker_quote_volume,alpha30,alpha36,alpha40,alpha45
str,datetime[ms],f64,f64,f64,f64,f64,datetime[ms],f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1000BONKUSDT""",2023-11-22 00:00:00,0.00478,0.004825,0.004076,0.004531,1.2701e10,2023-11-22 23:59:59.999,5.6368e7,320715,6.1849e9,2.7467e7,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.8900e7,null,null,null,null,-1.581977,-1.532579,1.365114,-0.643618
"""1000BONKUSDT""",2023-11-23 00:00:00,0.004531,0.004858,0.00393,0.004267,1.8971e10,2023-11-23 23:59:59.999,8.1590e7,573386,8.8982e9,3.8318e7,-5.826528,null,null,null,null,null,null,null,null,null,-5.826528,null,null,null,null,4.3272e7,null,null,null,null,-1.11916,-1.580292,1.422705,0.132825
"""1000BONKUSDT""",2023-11-24 00:00:00,0.004267,0.004335,0.003835,0.00414,1.7169e10,2023-11-24 23:59:59.999,6.9929e7,475254,7.9410e9,3.2393e7,-2.97633,-8.629442,null,null,null,null,null,null,null,null,-2.97633,null,null,null,null,3.7537e7,null,null,null,null,-1.11713,-1.543047,1.396197,0.25099
"""1000BONKUSDT""",2023-11-25 00:00:00,0.004143,0.0046,0.004084,0.00434,1.3093e10,2023-11-25 23:59:59.999,5.6988e7,501832,6.3891e9,2.7819e7,4.830918,1.710804,-4.215405,null,null,null,null,null,null,null,4.830918,null,null,null,null,2.9169e7,null,null,null,null,-1.130631,-1.588192,1.301208,0.011231
"""1000BONKUSDT""",2023-11-26 00:00:00,0.004341,0.004473,0.0039,0.004039,8.2273e9,2023-11-26 23:59:59.999,3.4347e7,344203,3.8247e9,1.5980e7,-6.935484,-2.439614,-5.343333,-10.85853,null,null,null,null,null,null,-6.935484,null,null,null,null,1.8366e7,null,null,null,null,-1.133583,-1.578087,1.004191,0.123405
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZRXUSDT""",2024-10-30 00:00:00,0.337,0.3454,0.3259,0.3387,3.3094e7,2024-10-30 23:59:59.999,1.1187e7,69215,1.6184e7,5.4729e6,0.534283,5.645664,5.382701,7.149636,6.075791,-0.877963,0.147842,-3.504274,-5.311714,-4.64527,0.534283,0.037009,0.03447,-0.590159,0.181498,5.7146e6,1.1342e8,1.2026e8,0.353856,2.3367e8,-0.326927,0.302397,-0.697516,-0.395288
"""ZRXUSDT""",2024-10-31 00:00:00,0.3388,0.3402,0.315,0.3175,2.6379e7,2024-10-31 23:59:59.999,8.6137e6,59773,1.2525e7,4.0909e6,-6.259226,-5.758385,-0.966937,-1.213441,0.442898,-0.563733,-7.082236,-6.120639,-9.54416,-11.238468,-6.259226,0.034517,0.032805,-0.413481,0.364253,4.5227e6,1.1515e8,1.2260e8,0.354262,2.3774e8,-1.31084,0.393852,-0.181491,-1.250315
"""ZRXUSDT""",2024-11-01 00:00:00,0.3176,0.3248,0.3059,0.3109,2.1759e7,2024-11-01 23:59:59.999,6.8310e6,54349,1.0382e7,3.2587e6,-2.07874,-8.207854,-7.717424,-3.025577,-3.266957,-1.645049,-2.630755,-9.013755,-8.072147,-11.424501,-2.07874,0.032877,0.032974,-0.380879,-0.244794,3.5723e6,1.1674e8,1.2447e8,0.35016,2.4121e8,-1.074798,0.342494,-0.367365,-1.388746


In [21]:
# below for combine factors
print(f"begin to calc linear compound factor: {FACTOR_COMBINATION_LIST}")
input_data = AddFutureRetCol(input_data, UPDATE_POSITION_TIME)
input_data

input_data = CalcLinearCompoundFactor(
    input_data, UPDATE_POSITION_TIME, FACTOR_COMBINATION_LIST
)

input_data, day_scale_df = AddTotalPosValueScale(
    input_data, day_num=10, trade_long_rank=20, trade_short_rank=10
)

if is_prod:
    # for normal run, save all data
    results = input_data
else:
    # for backtest and research, remove the last few rows and symbols whose min value is larger then 5 USDT
    results = input_data.filter(pl.col("close_price_fut_7day_ret").is_not_null())
results.write_parquet(output_path)
results

begin to calc linear compound factor: ['amihud', 'return_skewness', 'alpha30', 'alpha36', 'alpha40', 'alpha45', 'net_taker_quote_volume']
[ 1.36853096e+00 -2.94443704e-01  4.47032059e-01 -5.24699022e-16
 -1.00710642e+00  0.00000000e+00 -6.21442818e-01]
[ 6.29703025e-01 -8.59570195e-01 -3.07617469e-01 -5.08817881e-16
 -9.57668745e-01  0.00000000e+00 -6.95647464e-01]
[ 3.04302556e+00  5.93384438e-01 -6.23431973e-01 -5.49184829e-16
 -1.18903816e+00  0.00000000e+00 -1.32719621e-01]
[ 2.39805487e+00  2.75381413e+00 -1.22152370e+00  1.79876672e-15
  5.24623495e-01  0.00000000e+00  7.33686067e-01]
[4.62328752e+00 2.07963227e+00 3.20330677e-01 1.58472411e-15
 4.73441750e-01 0.00000000e+00 6.53557917e-01]
[ 5.02470179e+00  3.96346092e+00 -4.78534728e-01  7.97981602e-15
  9.43840061e-01  1.53308073e+00  1.99440715e+00]
[ 5.66018874e+00  3.98479068e+00  5.73408068e-01  6.82833600e-15
 -6.46072249e-01  6.73329276e-01  1.45888881e+00]
[-8.32465641e-01  5.77191411e+00 -4.00231756e-02  5.52747199e-15

symbol,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,past_1day_close_return,past_2day_close_return,past_3day_close_return,past_4day_close_return,past_5day_close_return,past_6day_close_return,past_7day_close_return,past_8day_close_return,past_9day_close_return,past_10day_close_return,return,open_price_volatility,close_price_volatility,amihud,return_skewness,taker_sell_quote_volume,taker_buy_quote_volume_sum,taker_sell_quote_volume_sum,net_taker_quote_volume,total_taker_quote_volume,alpha30,alpha36,alpha40,alpha45,close_price_fut_1day_ret,…,open_price_fut_7day_ret,close_price_fut_8day_ret,open_price_fut_8day_ret,close_price_fut_9day_ret,open_price_fut_9day_ret,close_price_fut_10day_ret,open_price_fut_10day_ret,linear_compound_factor_1day,linear_compound_factor_2day,linear_compound_factor_3day,linear_compound_factor_4day,linear_compound_factor_5day,linear_compound_factor_6day,linear_compound_factor_7day,linear_compound_factor_8day,linear_compound_factor_9day,linear_compound_factor_10day,long_value_scale_1day,short_value_scale_1day,long_value_scale_2day,short_value_scale_2day,long_value_scale_3day,short_value_scale_3day,long_value_scale_4day,short_value_scale_4day,long_value_scale_5day,short_value_scale_5day,long_value_scale_6day,short_value_scale_6day,long_value_scale_7day,short_value_scale_7day,long_value_scale_8day,short_value_scale_8day,long_value_scale_9day,short_value_scale_9day,long_value_scale_10day,short_value_scale_10day
str,datetime[ms],f64,f64,f64,f64,f64,datetime[ms],f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1000BONKUSDT""",2023-11-22 00:00:00,0.00478,0.004825,0.004076,0.004531,1.2701e10,2023-11-22 23:59:59.999,5.6368e7,320715,6.1849e9,2.7467e7,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.8900e7,null,null,null,null,-1.581977,-1.532579,1.365114,-0.643618,-5.826528,…,-13.493724,-14.257338,-14.60251,-11.013021,-18.74477,7.128669,-15.627615,null,null,null,null,null,null,null,null,null,null,1.2,0.8,0.8,1.2,0.8,1.2,1.2,0.8,1.0,1.0,1.0,1.0,0.8,1.2,1.0,1.0,1.0,1.0,0.8,1.2
"""1000BONKUSDT""",2023-11-23 00:00:00,0.004531,0.004858,0.00393,0.004267,1.8971e10,2023-11-23 23:59:59.999,8.1590e7,573386,8.8982e9,3.8318e7,-5.826528,null,null,null,null,null,null,null,null,null,-5.826528,null,null,null,null,4.3272e7,null,null,null,null,-1.11916,-1.580292,1.422705,0.132825,-2.97633,…,-9.909512,-5.507382,-14.279409,13.756738,-10.990951,24.67776,7.062459,null,null,null,null,null,null,null,null,null,null,1.0,1.0,1.2,0.8,0.8,1.2,0.8,1.2,1.2,0.8,1.2,0.8,1.2,0.8,0.8,1.2,1.0,1.0,1.0,1.0
"""1000BONKUSDT""",2023-11-24 00:00:00,0.004267,0.004335,0.003835,0.00414,1.7169e10,2023-11-24 23:59:59.999,6.9929e7,475254,7.9410e9,3.2393e7,-2.97633,-8.629442,null,null,null,null,null,null,null,null,-2.97633,null,null,null,null,3.7537e7,null,null,null,null,-1.11713,-1.543047,1.396197,0.25099,4.830918,…,-8.975861,17.246377,-5.483947,28.502415,13.686431,51.884058,24.724631,null,null,null,null,null,null,null,null,null,null,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.0,1.0,1.2,0.8,1.2,0.8,1.2,0.8,1.0,1.0,1.2,0.8
"""1000BONKUSDT""",2023-11-25 00:00:00,0.004143,0.0046,0.004084,0.00434,1.3093e10,2023-11-25 23:59:59.999,5.6988e7,501832,6.3891e9,2.7819e7,4.830918,1.710804,-4.215405,null,null,null,null,null,null,null,4.830918,null,null,null,null,2.9169e7,null,null,null,null,-1.130631,-1.588192,1.301208,0.011231,-6.935484,…,-2.655081,22.580645,17.089066,44.884793,28.457639,74.239631,51.774077,null,null,null,null,null,null,null,null,null,null,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8,1.0,1.0,1.2,0.8,1.2,0.8,1.2,0.8,1.2,0.8
"""1000BONKUSDT""",2023-11-26 00:00:00,0.004341,0.004473,0.0039,0.004039,8.2273e9,2023-11-26 23:59:59.999,3.4347e7,344203,3.8247e9,1.5980e7,-6.935484,-